In [1]:
pip install git+git://github.com/avinashkd44/keras.git --upgrade  # You will get here eigenvaluedecay regularizer for tensorflow backend. 

  Cloning git://github.com/avinashkd44/keras.git to /tmp/pip-req-build-xf17w53c
  Stored in directory: /tmp/pip-ephem-wheel-cache-6yxczoa4/wheels/06/40/64/3b72d52a4c1f9bbc237a7f506fde596520e0efa0e2c058ace2
Successfully built Keras
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [5]:
pip show keras # you can check keras version

Name: Keras
Version: 2.2.4
Summary: Deep Learning for humans
Home-page: https://github.com/keras-team/keras
Author: Francois Chollet
Author-email: francois.chollet@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras-preprocessing, scipy, h5py, keras-applications, numpy, six, pyyaml
Required-by: 


In [2]:
'''This script reuses pieces of code from the post:
"Building powerful image classification models using very little data"
from blog.keras.io
and from:
https://www.kaggle.com/tnhabc/state-farm-distracted-driver-detection/keras-sample
The training data can be downloaded at:
https://www.kaggle.com/c/state-farm-distracted-driver-detection/data
'''

import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import EigenvalueRegularizer
from numpy.random import permutation
from keras.optimizers import SGD
import pandas as pd
import datetime
import glob
import cv2
import math
import pickle
from collections import OrderedDict
from keras import backend as K
import tensorflow as tf
from keras.models import load_model
from keras.utils.conv_utils import convert_kernel
import keras.backend.tensorflow_backend as K
K.set_session
#from keras.utils.conv_utils import convert_kernel


Using TensorFlow backend.


<function keras.backend.tensorflow_backend.set_session>

In [3]:
from google.colab import drive      # you can mount your Google drive to use it as training/testing file location
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_data = '/content/gdrive/My Drive/datascience/train'    # locate your Data

In [0]:
train_data # you can check its loaded or not

'/content/gdrive/My Drive/datascience/train'

In [7]:
import os
os.listdir('/content/gdrive/My Drive/datascience/train')    # list your training folders

['c9', 'c1', 'c8', 'c7', 'c0', 'c4', 'c6', 'c2', 'c5', 'c3']

In [0]:

# Enter here the path to the model weights files:
weights_path = '/content/gdrive/My Drive/datascience/human action recog/vgg16_weights.h5'             # your vgg16_weights path

# Enter here the path to the top-model weights files:
top_model_weights_path = '/content/gdrive/My Drive/datascience/human action recog/fc_model.h5'        #your vgg16_model.h5path

# Enter here the path for storage of the whole model weights (VGG16+top classifier model):
whole_model_weights_path = '/content/gdrive/My Drive/datascience/human action recog/whole_model.h5'   # your whole_model.h5 path

# Enter here the name of the folder that contains the folders c0, c1,..., c9, with the training images belonging to classes 0 to 9:
train_data_dir = '/content/gdrive/My Drive/datascience/train'       # your training floder path

# Enter here the name of the folder where is the test images (the data evalueted in the private leaderboard):
test_data_dir = '/content/gdrive/My Drive/datascience/test'         # your test data path

test_images_path = '/content/gdrive/My Drive/datascience/test'      # your test image path could be same as test_data_dir

# Enter here the features of the data set:
img_width, img_height = 224, 224
nb_train_samples = 22424
nb_test_samples = 79726
color_type_global = 3

# You can set larger values here, according with the memory of your GPU:
batch_size = 32

# Enter here the number of training epochs (with 80 epochs the model was positioned among
# the 29% best competitors in the private leaderboard of state-farm-distracted-driver-detection)
# According to our results, this model can achieve a better performance if trained along a larger 
# number of epochs, due to the agressive regularization with Eigenvalue Decay that was adopted.
nb_epoch = 80

#Enter here the path for the whole model (VGG16+top classifier model):
#whole_model_weights_path = '/home/oswaldo/video_classification/competitionKaggle/whole_model.h5'


In [0]:
K.set_image_dim_ordering('tf')


In [10]:

# build the VGG16 network:
#model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="tf"))
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(img_height,img_width,3 )))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="tf"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="tf"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="tf"))



Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), strides=(2, 2), data_format="channels_last")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activatio

In [0]:
# loading the weights of the pre-trained VGG16:
# this is conversion from Theano backend to tensorflow


#assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
old = h5py.File(weights_path)
ops = []
for layer in model.layers:
   if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
      original_w = K.get_value(old)
      converted_w = convert_kernel(original_w)
      ops.append(tf.assign(old, converted_w).op)
      #model.layers[k].set_weights(weights)
K.get_session().run(ops)
model.save_weights('my_weights_tensorflow.h5') # loaded as tensorflow backend
old.close()
print('model loaded')
# now my new weight is my_weights_tensorflow.h5 of vgg16 model. so now repeat.
'''f = h5py.File(my_weights_tensorflow.h5)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')'''

model loaded


"f = h5py.File(my_weights_tensorflow.h5)\nfor k in range(f.attrs['nb_layers']):\n    if k >= len(model.layers):\n        break\n    g = f['layer_{}'.format(k)]\n    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]\n    model.layers[k].set_weights(weights)\nf.close()\nprint('Model loaded.')"

In [11]:
# Pooja look here - https://github.com/poojab5/keras/blob/master/keras/regularizers.py - line 110 - p is not defined. make it defined globaly
# https://github.com/avinashkd44/keras/blob/master/keras/regularizers.py   - mine


# building a classifier model on top of the convolutional model:

top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(64, activation='relu', W_regularizer=EigenvalueRegularizer(10)))
top_model.add(Dense(10, activation='softmax', W_regularizer=EigenvalueRegularizer(10)))
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model.add(top_model)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_regularizer=<keras.reg...)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_regularizer=<keras.reg...)`
  """


In [0]:
# setting the first 15 layers to non-trainable (the original weights will not be updated)
for layer in model.layers[:15]:
  layer.trainable = False

# Compiling the model with a SGD/momentum optimizer:

model.compile(loss = "categorical_crossentropy",
              optimizer=optimizers.SGD(lr=1e-6, momentum=0.9),
              metrics=['mean_squared_logarithmic_error', 'accuracy'])


In [13]:
# Data augmentation:

train_datagen = ImageDataGenerator(shear_range=0.3, zoom_range=0.3, rotation_range=0.3)
test_datagen = ImageDataGenerator()

print('trainning')
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')
  

print('testing')

trainning
Found 7704 images belonging to 10 classes.
testing


In [14]:
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

class_dictionary = train_generator.class_indices
sorted_class_dictionary = OrderedDict(sorted(class_dictionary.items()))
sorted_class_dictionary = sorted_class_dictionary.values()
print(sorted_class_dictionary)

Found 0 images belonging to 0 classes.
odict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [0]:
# Fine-tuning the model:  # it will take time
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=train_generator,
        nb_val_samples=nb_train_samples)
        
model.save_weights(whole_model_weights_path)

aux = model.predict_generator(test_generator, nb_test_samples)
predictions = np.zeros((nb_test_samples, 10))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=700, epochs=80, validation_steps=22424)`
  


Instructions for updating:
Use tf.cast instead.
Epoch 1/80
 15/700 [..............................] - ETA: 14:33:40 - loss: 14.7237 - mean_squared_logarithmic_error: 0.0437 - acc: 0.0125

In [0]:
# Rearranging the predictions:

ord = [5, 0, 6, 2, 7, 9, 1, 4, 8, 3]

for n in range(10):
    i = ord[n]
    print(i)
    print(aux[:, i])
    predictions[:, n] = aux[:, i]

# Trick to improve the multi-class logarithmic loss (the evaluation metric of state-farm-distracted-driver-detection from Keras):

predictions = 0.985 * predictions + 0.015


5


NameError: ignored

In [0]:
def get_im(path, img_width, img_height, color_type=1):
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_height, img_width))
    return resized


In [0]:
def load_test(img_width, img_height, color_type=1):
    print('Read test images')
    path = os.path.join(test_images_path, '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im(fl, img_width, img_height, color_type)
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1
        if total % thr == 0:
            print('Read {} images from {}'.format(total, len(files)))

    return X_test, X_test_id

X_test, test_id = load_test(img_width, img_height, color_type_global)


In [0]:
def create_submission(predictions, test_id):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

create_submission(predictions, test_id)
